In [7]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras import layers
from tensorflow.keras import models

In [8]:
# Define data directories
train_dir = "../data/train"
test_dir = "../data/test"

# Image dimensions and other parameters
img_width, img_height = 150, 150
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
# Generate batches of image data (train and validation)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Del notebook "SEarchCNNStructure" pudimos encontrar que la estructura de CNN óptima era la del Trial 63. A continuación se muestran los parámetros de la estructura óptima encontrada:

Trial 063 summary

Hyperparameters:

conv_filters_1: 16
conv_kernel_1: 3
num_conv_layers: 5
conv_filters_2: 64
conv_kernel_2: 3
num_dense_layers: 3
dense_units_1: 128
dense_units_2: 96
dense_units_3: 96
dense_units_4: 96
conv_filters_3: 64
conv_kernel_3: 5
conv_filters_4: 128
conv_kernel_4: 3
conv_filters_5: 96
conv_kernel_5: 3
conv_filters_6: 64
conv_kernel_6: 3
conv_filters_7: 64
conv_kernel_7: 5
Score: 0.5798272490501404

## CNN From scratch 

In [10]:
# Define the CNN model
# Vamos a tener: 6 capas convolucionales (intercaladas con capas MaxPooling) y 4 capas densas (la última con units = nºclases de los datos --> 7)

model = models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_width, img_height, 3), padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(64, (5,5), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(96, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(layers.Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding='same'))

# Flatten layer
model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(96, activation='relu'))
model.add(layers.Dense(96, activation='relu'))

# Output layer
model.add(layers.Dense(7, activation='softmax'))  # 7 output classes (emotions)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        9280      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 38, 64)        102464    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 19, 19, 64)       0

In [ ]:
# Compile the model
from keras import callbacks
num_epochs = 50
earlystopping = callbacks.EarlyStopping(monitor="val_accuracy",
                                        patience=5,
                                        restore_best_weights=True)


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[earlystopping]
)

model.save('model_orig.h5')

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("\nTest accuracy:", test_acc)

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.head()

In [ ]:
df_history['loss'].plot(legend=True)
df_history['val_loss'].plot(title='Losses', grid=True,
                              xlabel='Epochs',
                              ylabel='Loss',
                              legend=True)

In [ ]:
df_history['accuracy'].plot(legend=True)
df_history['val_accuracy'].plot(title='Accuracies', grid=True,
                              xlabel='Epochs',
                              ylabel='Accuracy',
                              legend=True)